In [6]:
import sys
#sys.path = [p for p in sys.path if p.find('/opt/apps/software/') == -1]
from glob import glob
from IPython.display import display, HTML

from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES
from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES

from utils.utils import read_all_datasets, transform_labels, create_directory, run_length_xps, generate_results_csv, plot_epochs_metric

from utils.data_loading import get_multiple_data,load_matlab, predict, shifted_zscore, zscore, print_metric, tf_rmse, tf_pmse, rmse

import utils

from classifiers import inception

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from itertools import compress

import numpy as np
import pandas as pd
import sklearn

def tf_pmse_DA(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=0)

def tf_pmse_5HT(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=1)

def tf_pmse_pH(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=2)

def tf_pmse_NE(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=3)

# import keras
# keras.backend.tensorflow_backend._get_available_gpus()

In [35]:
data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-A-RBE-97Hz/'
voltammograms = glob(data_prefix + '/*MMA*/voltammograms.mat')
probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04', 'MMA018W01R04', 'MMA019W01R04', 'MMA022W01R04', 'MMA023W01R04', 'MMA024W01R04', 'MMA025W01R04']
good_probes = probes

hold_probe = probes.pop(3)
print('hold probe :', hold_probe)

# val_probe=None
val_probe = probes.pop(5)

print(f'Leaving out probe {hold_probe}', flush=True)
print(f'Validation probe {val_probe}', flush=True)
print(f'Loading data', flush=True)

filter_files = lambda x : x.find('500kohm') == -1
print('No 500ohms')

# filter_files = lambda x : x.find('500kohm') > -1
# print('ONLY 500ohms')

# normalize_data = minmax
# revert_data = lambda x: minmax(x, inverse=True)

# normalize_data = shifted_zscore
# revert_data = lambda x: shifted_zscore(x, inverse=True)

normalize_data = shifted_zscore
revert_data = lambda x: shifted_zscore(x, inverse=True)

# normalize_data = lambda x: x
# revert_data = lambda x: x

# nrecords_per_session = 10
nrecords_per_session = 300
# nrecords_per_session = 1

voltammograms = glob(data_prefix + '/*MMA*/voltammograms.mat')
print('number of voltammograms files %d'%len(voltammograms))
voltammograms = list(compress(voltammograms, [filter_files(x) for x in voltammograms]))
print('after filter %d'%len(voltammograms))

holdout_files = list(compress(voltammograms, [x.find(hold_probe) > -1 for x in voltammograms]))
print('number of holdout files %d'%len(holdout_files))
x_test, y_test = load_matlab(holdout_files, nrecords_per_session, asnumpy=True, proj_y=normalize_data)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
voltammograms = list(compress(voltammograms, [x.find(hold_probe) == -1 for x in voltammograms]))

val_files = list(compress(voltammograms, [x.find(val_probe) > -1 for x in voltammograms]))
print('number of validation files %d'%len(val_files))
x_val, y_val = load_matlab(val_files, nrecords_per_session, asnumpy=True, proj_y=normalize_data)
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

voltammograms = list(compress(voltammograms, [x.find(val_probe) == -1 for x in voltammograms]))
print('number of train files %d'%len(voltammograms))

x_train = {probe: [] for probe in probes}
y_train = {probe: [] for probe in probes}
for probe in probes:
    probe_files = list(compress(voltammograms, [x.find(probe) > -1 for x in voltammograms]))
    print('number of %s files %d'%(probe,len(probe_files)))
    x_train[probe], y_train[probe] = load_matlab(probe_files, nrecords_per_session, asnumpy=True, proj_y=normalize_data)
    x_train[probe] = x_train[probe].reshape((x_train[probe].shape[0], x_train[probe].shape[1], 1))
    voltammograms = list(compress(voltammograms, [x.find(probe) == -1 for x in voltammograms]))
    
print('any files left? %d'%len(voltammograms))

print('Data loaded')

output_shape = y_train[probes[0]].shape[1]


hold probe : MMA018W01R04
Leaving out probe MMA018W01R04
Validation probe MMA023W01R04
Loading data
No 500ohms
number of voltammograms files 36
after filter 36
number of holdout files 4
(39600, 999)
(39600, 4)
number of validation files 4
(39600, 999)
(39600, 4)
number of train files 28
number of MMA003W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA004W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA013W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA019W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA022W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA024W01R04 files 4
(39600, 999)
(39600, 4)
number of MMA025W01R04 files 4
(39600, 999)
(39600, 4)
any files left? 0
Data loaded


In [36]:
output_directory = f'/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm/{val_probe}/'

if not (os.path.exists(output_directory)):
    os.makedirs(output_directory, exist_ok=True)


In [51]:
# resnet model
import keras
import numpy as np
import time

from utils.utils import save_logs
from utils.utils import calculate_metrics
from utils.utils import save_test_duration

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def tf_rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

class Regression_INCEPTION:

    def __init__(self, output_directory, input_shape, output_shape, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=100,
                pre_model=None):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape, output_shape, pre_model=pre_model)
#             if (verbose == True):
#                 self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

            
    def _calculate_metrics(self, y_true, y_pred, duration):
        res = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                           columns=['rmse_DA', 'rmse_5HT', 'rmse_pH', 'rmse_NE', 'duration'])
        y_pred = np.apply_along_axis(revert_data, axis=1, arr=y_pred) 
        y_true = np.apply_along_axis(revert_data, axis=1, arr=y_true) 
#         rmse(y_true, y_pred, multioutput='uniform_average')
        rmse4 = rmse(y_true, y_pred)
        res['rmse_DA'] = rmse4[0]
        res['rmse_5HT'] = rmse4[1]
        res['rmse_pH'] = rmse4[2]
        res['rmse_NE'] = rmse4[3]
        res['duration'] = duration
        return res

    def _save_logs(self, hist, y_pred, y_true, duration,
                  lr=True, plot_test_acc=True):
        hist_df = pd.DataFrame(hist.history)
        hist_df.to_csv(self.output_directory + 'history.csv', index=False)

        df_metrics = self._calculate_metrics(y_true, y_pred, duration)
        df_metrics.to_csv(self.output_directory + 'df_metrics.csv', index=False)

        index_best_model = hist_df['loss'].idxmin()
        row_best_model = hist_df.loc[index_best_model]

        df_best_model = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                                     columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_learning_rate', 'best_model_nb_epoch'])

        df_best_model['best_model_train_loss'] = row_best_model['loss']
        if plot_test_acc:
            df_best_model['best_model_val_loss'] = row_best_model['val_loss']
#         df_best_model['best_model_train_acc'] = row_best_model['acc']
#         if plot_test_acc:
#             df_best_model['best_model_val_acc'] = row_best_model['val_acc']
        if lr == True:
            df_best_model['best_model_learning_rate'] = row_best_model['lr']
        df_best_model['best_model_nb_epoch'] = index_best_model

        df_best_model.to_csv(self.output_directory + 'df_best_model.csv', index=False)

        if plot_test_acc:
            # plot losses
            plot_epochs_metric(hist, self.output_directory + 'epochs_loss.png')
            plot_epochs_metric(hist, self.output_directory + 'epochs_DA.png', metric='tf_pmse_DA')
            plot_epochs_metric(hist, self.output_directory + 'epochs_5HT.png', metric='tf_pmse_5HT')
            plot_epochs_metric(hist, self.output_directory + 'epochs_pH.png', metric='tf_pmse_pH')
            plot_epochs_metric(hist, self.output_directory + 'epochs_NE.png', metric='tf_pmse_NE')

        return df_metrics

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.normalization.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, output_shape, pre_model=None):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(output_shape, activation='softplus')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)
        
        if not pre_model is None:
            print('loading previous weights...')
            for i in range(len(model.layers)-1):
                model.layers[i].set_weights(pre_model.layers[i].get_weights())
        else:
            print('starting model from scratch...')

#         model.compile(loss='mse', optimizer=keras.optimizers.Adam(), metrics=[])
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(), metrics=[tf_pmse_DA, tf_pmse_5HT, tf_pmse_pH, tf_pmse_NE])
#         model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9), metrics=[tf_pmse_DA, tf_pmse_5HT, tf_pmse_pH, tf_pmse_NE])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, plot_test_acc=False):
        if len(keras.backend.tensorflow_backend._get_available_gpus()) == 0:
            print('error no gpu')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 20, 16))
        else:
            mini_batch_size = self.batch_size
        print(f'mini batch size: {mini_batch_size}')

        start_time = time.time()

        if plot_test_acc:
            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)
        else:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        print('predicting test set...', end = '')
        y_pred = self.predict(x_val, y_val, x_train, y_train, return_df_metrics=False)
        print(' done.')

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        df_metrics = self._save_logs(hist, y_pred, y_val, duration, plot_test_acc=plot_test_acc)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_test, x_train, y_train, return_df_metrics=True):
        start_time = time.time()
        model = self.get_best_model()
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            df_metrics = self._calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

    def get_best_model(self):
        model_path = self.output_directory + 'best_model.hdf5'
        return keras.models.load_model(model_path, 
                                        custom_objects={"tf_pmse_DA": tf_pmse_DA, "tf_pmse_5HT": tf_pmse_5HT, "tf_pmse_pH": tf_pmse_pH, "tf_pmse_NE": tf_pmse_NE})        

In [53]:
# Fit the model

classifier_name = 'inception'

input_shape = x_train[probes[0]].shape[1:]

pre_model = None
nprobe = len(probes)
for (iprobe, probe) in enumerate(probes):
    print(f'probe {probe} ({iprobe} out of {nprobe})')
    classifier = Regression_INCEPTION(output_directory, input_shape, output_shape, batch_size=128, verbose=1, build=True, nb_epochs=100, pre_model=pre_model)
    metrics = classifier.fit(x_train[probe], y_train[probe], x_val, y_val, plot_test_acc=True)
    display(HTML(metrics.to_html()))
    pre_model = classifier.get_best_model()


probe MMA003W01R04 (0 out of 7)
starting model from scratch...
mini batch size: 128
Train on 39600 samples, validate on 39600 samples
Epoch 1/100
39600/39600 [==============================] - 51s 1ms/step - loss: 7.0142 - tf_pmse_DA: 1827.3674 - tf_pmse_5HT: 1812.7809 - tf_pmse_pH: 0.5480 - tf_pmse_NE: 2027.3330 - val_loss: 5.0274 - val_tf_pmse_DA: 2147.0379 - val_tf_pmse_5HT: 2083.6658 - val_tf_pmse_pH: 0.4439 - val_tf_pmse_NE: 2649.2678
Epoch 2/100
39600/39600 [==============================] - 39s 986us/step - loss: 0.5885 - tf_pmse_DA: 832.5015 - tf_pmse_5HT: 782.9627 - tf_pmse_pH: 0.2407 - tf_pmse_NE: 762.3323 - val_loss: 25.8897 - val_tf_pmse_DA: 1174.9227 - val_tf_pmse_5HT: 1940.1833 - val_tf_pmse_pH: 3.1232 - val_tf_pmse_NE: 882.8029
Epoch 3/100
39600/39600 [==============================] - 39s 982us/step - loss: 0.4307 - tf_pmse_DA: 799.9527 - tf_pmse_5HT: 722.9886 - tf_pmse_pH: 0.1258 - tf_pmse_NE: 718.2580 - val_loss: 20.5323 - val_tf_pmse_DA: 2554.4718 - val_tf_pmse_5HT: 

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1658.576136,2039.686252,0.401139,1832.814921,3919.406573


probe MMA004W01R04 (1 out of 7)
loading previous weights...
mini batch size: 128
Train on 39600 samples, validate on 39600 samples
Epoch 1/100
39600/39600 [==============================] - 45s 1ms/step - loss: 9.3755 - tf_pmse_DA: 2033.6034 - tf_pmse_5HT: 2275.4397 - tf_pmse_pH: 0.4863 - tf_pmse_NE: 2160.5125 - val_loss: 4.7905 - val_tf_pmse_DA: 2166.0134 - val_tf_pmse_5HT: 2021.9498 - val_tf_pmse_pH: 0.5715 - val_tf_pmse_NE: 2157.4599
Epoch 2/100
39600/39600 [==============================] - 37s 936us/step - loss: 0.3345 - tf_pmse_DA: 700.2191 - tf_pmse_5HT: 667.8745 - tf_pmse_pH: 0.0958 - tf_pmse_NE: 609.0535 - val_loss: 1.3822 - val_tf_pmse_DA: 874.6271 - val_tf_pmse_5HT: 1283.6424 - val_tf_pmse_pH: 0.2861 - val_tf_pmse_NE: 1077.0526
Epoch 3/100
39600/39600 [==============================] - 37s 933us/step - loss: 0.1333 - tf_pmse_DA: 461.7793 - tf_pmse_5HT: 421.1275 - tf_pmse_pH: 0.0644 - tf_pmse_NE: 351.0016 - val_loss: 3.4322 - val_tf_pmse_DA: 1067.6676 - val_tf_pmse_5HT: 1281.

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1526.84529,1840.709853,0.710929,1870.820653,3707.307184


probe MMA013W01R04 (2 out of 7)
loading previous weights...
mini batch size: 128
Train on 39600 samples, validate on 39600 samples
Epoch 1/100
39600/39600 [==============================] - 44s 1ms/step - loss: 10.5874 - tf_pmse_DA: 1861.7196 - tf_pmse_5HT: 2163.0486 - tf_pmse_pH: 0.6862 - tf_pmse_NE: 2175.0361 - val_loss: 2.9312 - val_tf_pmse_DA: 864.8962 - val_tf_pmse_5HT: 874.5574 - val_tf_pmse_pH: 0.8493 - val_tf_pmse_NE: 862.6086
Epoch 2/100
39600/39600 [==============================] - 38s 953us/step - loss: 0.3518 - tf_pmse_DA: 679.9763 - tf_pmse_5HT: 668.5079 - tf_pmse_pH: 0.1036 - tf_pmse_NE: 691.1841 - val_loss: 2.3809 - val_tf_pmse_DA: 1469.2769 - val_tf_pmse_5HT: 1059.3702 - val_tf_pmse_pH: 0.2807 - val_tf_pmse_NE: 1646.7005
Epoch 3/100
39600/39600 [==============================] - 37s 941us/step - loss: 0.2003 - tf_pmse_DA: 540.2955 - tf_pmse_5HT: 488.8423 - tf_pmse_pH: 0.0823 - tf_pmse_NE: 497.4655 - val_loss: 5.6662 - val_tf_pmse_DA: 2875.5683 - val_tf_pmse_5HT: 835.68

...

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1171.642633,1355.284705,0.724602,1719.031366,3709.88972


probe MMA024W01R04 (5 out of 7)
loading previous weights...
mini batch size: 128
Train on 39600 samples, validate on 39600 samples
Epoch 1/100
39600/39600 [==============================] - 45s 1ms/step - loss: 6.3463 - tf_pmse_DA: 1642.3681 - tf_pmse_5HT: 1420.2836 - tf_pmse_pH: 0.4500 - tf_pmse_NE: 2121.9984 - val_loss: 4.3344 - val_tf_pmse_DA: 892.2205 - val_tf_pmse_5HT: 1116.9659 - val_tf_pmse_pH: 1.0437 - val_tf_pmse_NE: 1172.6672
Epoch 2/100
39600/39600 [==============================] - 37s 931us/step - loss: 0.3613 - tf_pmse_DA: 683.4097 - tf_pmse_5HT: 706.6646 - tf_pmse_pH: 0.1247 - tf_pmse_NE: 640.5417 - val_loss: 3.8002 - val_tf_pmse_DA: 1081.9174 - val_tf_pmse_5HT: 965.8120 - val_tf_pmse_pH: 0.9111 - val_tf_pmse_NE: 1272.1042
Epoch 3/100
39600/39600 [==============================] - 37s 937us/step - loss: 0.2017 - tf_pmse_DA: 519.5278 - tf_pmse_5HT: 518.2019 - tf_pmse_pH: 0.0862 - tf_pmse_NE: 488.9188 - val_loss: 3.3957 - val_tf_pmse_DA: 948.0811 - val_tf_pmse_5HT: 1750.04

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1864.253068,1497.516636,0.461606,2241.278884,3717.289772


probe MMA025W01R04 (6 out of 7)
loading previous weights...
mini batch size: 128
Train on 39600 samples, validate on 39600 samples
Epoch 1/100
39600/39600 [==============================] - 45s 1ms/step - loss: 9.1827 - tf_pmse_DA: 2284.5365 - tf_pmse_5HT: 1679.4013 - tf_pmse_pH: 0.6012 - tf_pmse_NE: 2129.5945 - val_loss: 3.4114 - val_tf_pmse_DA: 1152.3713 - val_tf_pmse_5HT: 1147.9427 - val_tf_pmse_pH: 0.7274 - val_tf_pmse_NE: 1530.1415
Epoch 2/100
39600/39600 [==============================] - 37s 932us/step - loss: 0.4604 - tf_pmse_DA: 792.0633 - tf_pmse_5HT: 766.4931 - tf_pmse_pH: 0.1546 - tf_pmse_NE: 703.8063 - val_loss: 4.9104 - val_tf_pmse_DA: 1055.0861 - val_tf_pmse_5HT: 1273.5073 - val_tf_pmse_pH: 1.1441 - val_tf_pmse_NE: 1431.0962
Epoch 3/100
39600/39600 [==============================] - 36s 920us/step - loss: 0.3481 - tf_pmse_DA: 725.2495 - tf_pmse_5HT: 695.8132 - tf_pmse_pH: 0.1045 - tf_pmse_NE: 604.3104 - val_loss: 4.5792 - val_tf_pmse_DA: 1732.8955 - val_tf_pmse_5HT: 2031

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1807.374944,2493.649692,0.519145,1265.003391,3717.074324


In [ ]:
hold_conc_pred = predict(invert, vae, hold_data)

In [15]:
y_pred = np.load('/mnt/nfs/proj/in-vitro/Leonardo/inception/results/inception/TSC/Meat/y_pred.npy')

In [26]:
acc = np.mean(y_true==np.argmax(y_pred,axis=1))

In [27]:
acc

0.9333333333333333

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras

In [7]:
tf.keras.metrics.RootMeanSquaredError

tensorflow.python.keras.metrics.RootMeanSquaredError

In [7]:
keras.__version__

'2.2.4'